In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from skimage import io
from PIL import Image
from glob import glob
from tqdm import tqdm
from sklearn.cluster import KMeans

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *

In [ ]:
train_path='dataset1/train'

In [ ]:
train_images=[]
train_masks=[]
def load_images(path):
    temp_img,temp_mask=[],[]
    images=glob(os.path.join(path,'*.jpg'))
    for i in tqdm(images):
        i=cv2.imread(i)
        #i=cv2.normalize(i,None,0,1,cv2.NORM_MINMAX,cv2.CV_32F)
        img=i[:,:256]
        msk=i[:,256:]  
        temp_img.append(img)
        temp_mask.append(msk)
    return temp_img,temp_mask

train_images,train_masks=load_images(train_path)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(15,5))
axes[0].imshow(train_images[10])
axes[1].imshow(train_masks[10])

In [ ]:
train_images = np.array(train_images)
train_masks = np.array(train_masks)

In [ ]:
from sklearn.model_selection import train_test_split

trainx, testx, trainy, testy = train_test_split(train_images,train_masks, test_size=0.25, random_state=40)
print(trainx.shape, testx.shape, trainy.shape, testy.shape)

In [ ]:
def U_net():
        # Build U-Net model
        inputs = Input((256, 256, 3))
        s = Lambda(lambda x: x / 255) (inputs)
        
        c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (s)
        c1 = Dropout(0.1) (c1)
        c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c1)
        p1 = MaxPooling2D((2, 2)) (c1)

        c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
        c2 = Dropout(0.1) (c2)
        c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
        p2 = MaxPooling2D((2, 2)) (c2)

        c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
        c3 = Dropout(0.2) (c3)
        c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
        p3 = MaxPooling2D((2, 2)) (c3)

        c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
        c4 = Dropout(0.2) (c4)
        c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c4)
        p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

        c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
        c5 = Dropout(0.3) (c5)
        c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c5)

        u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
        u6 = concatenate([u6, c4])
        c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u6)
        c6 = Dropout(0.2) (c6)
        c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c6)

        u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
        u7 = concatenate([u7, c3])
        c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u7)
        c7 = Dropout(0.2) (c7)
        c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c7)

        u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
        u8 = concatenate([u8, c2])
        c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u8)
        c8 = Dropout(0.1) (c8)
        c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c8)

        u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
        u9 = concatenate([u9, c1], axis=3)
        c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
        c9 = Dropout(0.1) (c9)
        c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c9)

        outputs = Conv2D(19, (1, 1), activation='softmax') (c9)
        model = Model(inputs=[inputs], outputs=[outputs])
        model.summary()
        
        return model

In [ ]:
model = U_net()

In [ ]:
plt.imshow(trainx[1])

In [ ]:
plt.imshow(trainy[1])

In [ ]:
def dice_coefficient(y_true,y_pred):
    numerator = 2*tf.reduce_sum(y_true*y_pred)
    denominator = tf.reduce_sum(y_true+y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())

In [ ]:
def loss(y_true,y_pred):
    return tf.keras.losses.sparse_categorical_crossentropy(y_true,y_pred) - tf.math.log(dice_coefficient(y_true,y_pred) + tf.keras.backend.epsilon())

In [ ]:
callbacks=[
    ModelCheckpoint("unet.h5", verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger("check.csv"),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False)
]

In [ ]:
opt=Adam(learning_rate=1e-3)
loss = tf.keras.losses.sparse_categorical_crossentropy
model.compile(optimizer=opt, loss = loss, metrics=['accuracy'])

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
ntrainy = to_categorical(np.asarray(trainy))

In [ ]:
ntrainy[0].shape

In [ ]:
model.fit(trainx, to_categorical(np.asarray(trainy), num_classes=19), batch_size=64, epochs = 100, validation_data = (testx, to_categorical(np.asarray(testy), num_classes=19)), callbacks=callbacks)

In [ ]:
pred=model.predict(testx)
y_pred=tf.argmax(pred,axis=-1)

In [ ]:
def show_predications(x_test,y_test,y_pred):
    idx=np.random.randint(0,len(y_pred))
    fig, axes = plt.subplots(1,3,figsize=(10,10))
    axes[0].imshow(x_test[idx])
    axes[0].set_title("original")
    axes[1].imshow(y_test[idx])
    axes[1].set_title("mask")
    axes[2].imshow(y_pred[idx])
    axes[2].set_title("predicated")

In [ ]:
for i in range(3):
    show_predications(testx,testy,y_pred)

In [ ]:
plt.imshow(testy[0])